In [3]:
import time
import threading
import json
import os
import datetime
import configparser
from dataclasses import dataclass

from binance.spot import Spot as Client
from binance.lib.utils import config_logging
from binance.error import ClientError

import pandas as pd
import numpy as np
from catboost import CatBoostClassifier

from save_earnings import addEarnings
from ipynb.fs.defs.get_hours import get_last_hours

import math

In [28]:
import os
from dotenv import load_dotenv
dotenv_path = '.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path)

In [29]:
@dataclass
class ModelParametres:
    model_type: str
    symbol: str
    time_interval: str
    time_intervals_for_prediction: int
    percent: float
    precision: int
    price_precision: int
    loss_percent: float 


In [61]:
models = []
models.append(ModelParametres('catboost', 'BTC', '1m', 30, 1.005, 4, 1, 0.996))
models.append(ModelParametres('catboost', 'SPELL', '1m', 30, 1.01, 0, 6, 0.996))

models.append(ModelParametres('catboost', 'CAKE', '5m', 30, 1.02, 2, 2, 0.989))
models.append(ModelParametres('catboost', 'DOGE', '5m', 30, 1.02, 0, 4, 0.989))

models.append(ModelParametres('catboost', 'SHIB', '1m', 30, 1.01, 0, 7, 0.996))
models.append(ModelParametres('catboost', 'ALPACA', '1m', 30, 1.01, 1, 3, 0.996))
models.append(ModelParametres('catboost', 'SCRT', '1m', 30, 1.01, 1, 2, 0.996))
models.append(ModelParametres('catboost', 'CRV', '1m', 30, 1.01, 1, 2, 0.996))

In [2]:
api_key = os.environ.get('API_KEY')
api_secret = os.environ.get('API_SECRET')
client = Client(api_key, api_secret)

client_mutex = threading.Lock()

In [8]:
#client.exchange_info('CRVUSDT')

{'timezone': 'UTC',
 'serverTime': 1661421972015,
 'rateLimits': [{'rateLimitType': 'REQUEST_WEIGHT',
   'interval': 'MINUTE',
   'intervalNum': 1,
   'limit': 1200},
  {'rateLimitType': 'ORDERS',
   'interval': 'SECOND',
   'intervalNum': 10,
   'limit': 50},
  {'rateLimitType': 'ORDERS',
   'interval': 'DAY',
   'intervalNum': 1,
   'limit': 160000},
  {'rateLimitType': 'RAW_REQUESTS',
   'interval': 'MINUTE',
   'intervalNum': 5,
   'limit': 6100}],
 'exchangeFilters': [],
 'symbols': [{'symbol': 'CRVUSDT',
   'status': 'TRADING',
   'baseAsset': 'CRV',
   'baseAssetPrecision': 8,
   'quoteAsset': 'USDT',
   'quotePrecision': 8,
   'quoteAssetPrecision': 8,
   'baseCommissionPrecision': 8,
   'quoteCommissionPrecision': 8,
   'orderTypes': ['LIMIT',
    'LIMIT_MAKER',
    'MARKET',
    'STOP_LOSS_LIMIT',
    'TAKE_PROFIT_LIMIT'],
   'icebergAllowed': True,
   'ocoAllowed': True,
   'quoteOrderQtyMarketAllowed': True,
   'allowTrailingStop': True,
   'cancelReplaceAllowed': True,
   

In [42]:
def getCoinBalance(client):
    client_mutex.acquire()
    balance = client.user_asset()
    client_mutex.release()
    return balance

#Market buy
def executeBuy(client, qtyBuy, symbol):
    client_mutex.acquire()
    print('mutex acquired')
    print(f'{qtyBuy} is {symbol}')
    params = {
        "symbol": symbol + 'USDT',
        "side": "BUY",
        "type": "MARKET",
        #"timeInForce": "GTC",
        "quantity": qtyBuy
    }
    print('sending request')
    try: 
        response = client.new_order(**params)
    except Exception as e:
        print(e)
    print('releasing mutex')
    client_mutex.release()
    return response

#Market sell
def executeSell(client, qtySell, symbol, orderType, stopPrice = None, timeInForce = None):
    client_mutex.acquire()
    print('mutex acquired')
    print(f'{qtySell} is {symbol}')
    params = {
        "symbol": symbol + 'USDT',
        "side": "SELL",
        "type": orderType,
        "quantity": qtySell
    }
    if stopPrice:
        params['stopPrice'] = stopPrice
        params['price'] = stopPrice
    if timeInForce:
        params['timeInForce'] = timeInForce
    print(f'sending request with params {params}')
    try: 
        response = client.new_order(**params)
    except Exception as e:
        print(e)
    print('releasing mutex')
    client_mutex.release()
    return response

def newOcoOrder(client, quantity, symbol, price, stopPrice, stopLimitPrice, timeInForce):
    client_mutex.acquire()
    params = {
        "symbol": symbol + 'USDT',
        "side": "SELL",
        "quantity": quantity,
        "price": price,
        "stopPrice": stopPrice,
        "stopLimitPrice": stopLimitPrice,
        "stopLimitTimeInForce": timeInForce,
    }
    print(f'sending request with params {params}')
    try: 
        response = client.new_oco_order(**params)
    except Exception as e:
        print(e)
    print('releasing mutex')
    client_mutex.release()
    return response

def getTickerPrice(client, symbol):
    client_mutex.acquire()
    
    tick_price = float(client.ticker_price(symbol+'USDT')['price'])

    client_mutex.release()
    
    return tick_price

def check_order_status(client, symbol, orderId):
    client_mutex.acquire()

    result = client.get_order(symbol + 'USDT', orderId=orderId)['status'] == 'FILLED'
    
    client_mutex.release()

    return result

def get_order_price(client, symbol, orderId):
    client_mutex.acquire()

    result = float(client.get_order(symbol + 'USDT', orderId=orderId)['price'])

    client_mutex.release()

    return result

def cancel_order(client, symbol, orderId):
    client_mutex.acquire()

    client.cancel_order(symbol + 'USDT', orderId=orderId)

    client_mutex.release()


In [38]:
def main_bot_function(parametres: ModelParametres):
    if parametres.model_type == 'catboost':
        model = CatBoostClassifier()
    symbol = parametres.symbol
    time_interval = parametres.time_interval
    model.load_model(f"true_models/{parametres.model_type}_{symbol}_{time_interval}")
    percent = parametres.percent
    time_intervals_for_prediction = parametres.time_intervals_for_prediction
    precision = parametres.precision
    price_precision = parametres.price_precision
    loss_percent = parametres.loss_percent

    state = 'waiting'

    print('Waiting (Porosad)')

    current_amount = 0
    time_waiting = 0
    
    now = datetime.datetime.now()
    time.sleep(60 - now.second)
    candle_interval = time_interval[-1]
    if candle_interval == 'm':
        candle_interval = 1
        time.sleep((int(time_interval[:-1]) - now.minute % int(time_interval[:-1])) * 60)
    elif candle_interval == 'h':
        candle_interval = 60
    elif candle_interval == 'd':
        candle_interval = 60 * 24
    candle_interval *= int(time_interval[:-1]) * 60
    
    while True:
        if state == 'waiting':
            min_value = round(25 / getTickerPrice(client, symbol), precision)
            if precision <= 0:
                min_value = int(min_value)
            client_mutex.acquire()
            current_data = get_last_hours(symbol, time_interval, time_intervals_for_prediction, client)
            client_mutex.release()
            if model.predict(current_data)[0] == 1:
                time_waiting = 0
                print('Trying to buy.')
                order = executeBuy(client, min_value, symbol)
                print(order)
                bought_price = float(order['fills'][0]['price'])
                print(f'Bought price is: {bought_price}.')
                time.sleep(3)
                flag = False
                print('checking balance')
                amount = getCoinBalance(client)
                for money in amount:
                    if money['asset'] == symbol:
                        flag = True
                        current_amount = math.trunc(float(money['free']) * (10 ** precision)) / (10 ** precision)
                        if precision <= 0:
                            current_amount = int(current_amount)
                if not flag:
                    print('something go wrong with buy!!!!!!!')
                    continue
                state = 'bought'
                time_waiting = 0
                print(f'Bought {current_amount} {symbol} take_profit - {round(bought_price * percent, price_precision)} \
                            stop loss - { round(bought_price * loss_percent, price_precision)}')
                print('Waiting (GigaChad)')
                ocoOrder = newOcoOrder(client, quantity=current_amount, symbol=symbol, 
                                    price=round(bought_price * percent, price_precision), 
                                    stopPrice= round(bought_price * loss_percent, price_precision),
                                    stopLimitPrice = round(bought_price * loss_percent, price_precision),
                                    timeInForce='GTC')
                take_profit_order_id = ocoOrder['orderReports'][0]['orderId']
                stop_loss_order_id = ocoOrder['orderReports'][1]['orderId']
                if ocoOrder['orderReports'][0]['type'] == 'STOP_LOSS_LIMIT':
                    take_profit_order_id, stop_loss_order_id = stop_loss_order_id, take_profit_order_id
            else:
                print(f'Waiting {time_waiting // 60}m already (PoroSad).')
                time_waiting += candle_interval
                time.sleep(candle_interval)
        elif state == 'bought':
            message = ''
            if check_order_status(client, symbol, take_profit_order_id):
                message = 'take_profit'
                current_price = get_order_price(client, symbol, take_profit_order_id)
            elif check_order_status(client, symbol, stop_loss_order_id):
                message = 'stop_loss'
                current_price = get_order_price(client, symbol, stop_loss_order_id)
            else:
                if time_waiting % 10 == 0:
                    print(f'Waiting {time_waiting // 60}m already (GigaChad).')
                time_waiting += candle_interval
                time.sleep(candle_interval)
                continue
            current_price *= 0.999
            time_waiting = 0
            state = 'waiting'
            print(f'{message} worked.')
            print(f'earned {(current_price - bought_price) * current_amount}   USDT!')
            print(f'{current_amount} {symbol}. ({bought_price} - {current_price}) {message} worked. Profit {(current_price - bought_price) * current_amount} USDT!')
            addEarnings((current_price - bought_price) * current_amount)
            print('Waiting (PoroSad)')
            now = datetime.datetime.now()
            time.sleep(60 - now.second)
            if time_interval[-1] == 'm':
                time.sleep((int(time_interval[:-1]) - now.minute % int(time_interval[:-1])) * 60)

In [ ]:
threads = []
for model in models:
    threads.append(threading.Thread(target=main_bot_function, args=[model]))
    threads[-1].start()
    print(f'Started model {model.model_type}_{model.symbol}_{model.time_interval}')
    time.sleep(3)

In [ ]:
while True:
    time.sleep(60 * 60 * 24 * 385)